In [3]:
import pickle
import time
import numpy as np
import pandas as pd
import os
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from nltk.corpus import stopwords
from pylab import *
from bs4 import BeautifulSoup
import random
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.corpus import stopwords 
import string
from nltk.tag import pos_tag
from sklearn import metrics
#from textblob import TextBlob as tb
from collections import OrderedDict

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#from nltk.corpus import stopwords
#stop_words = set(stopwords.words('english'))
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anchi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
def tf(word, blob):
    return blob.words.count(word) / len(blob.noun_phrases)#.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.noun_phrases)#.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

def tokenize(text):
    text= str(text).lower()    
    text = text.replace("'", "")          
    text = re.sub('(\d{1,4})[/.-:](\d{1,2})[/.-:](\d{0,4})', ' ', text).strip()      
    text = re.sub(r"\b[-+]?[\d.]+\b", ' ', text)    
    text = re.sub('[^A-Za-z0-9]+', ' ', text)        
#     words = [word for word in nltk.word_tokenize(text)] #for sent in nltk.sent_tokenize(text)      
#     words = [w for w in words if w not in stop_words and len(w)>=2 ]          
#     words = [stemmer.stem(i) for i in words]   
    return text#words

In [5]:
def build_LDA_Model(tfs):
    lda_model = LatentDirichletAllocation(n_topics=8,               # Number of topics
                                          max_iter=10,               
    # Max learning iterations
                                          learning_method='online',   
                                          random_state=100,          
    # Random state
                                          batch_size=128,            
    # n docs in each learning iter
                                          evaluate_every = -1,       
    # compute perplexity every n iters, default: Don't
                                          n_jobs = -1,               
    # Use all available CPUs
                                         )
    lda_output = lda_model.fit_transform(tfs)
    print(lda_model)  # Model attributes
    
    #Analyzed model performance with per

In [7]:
def classify_doc_topics(docnames,tfs, feature_names,best_lda_model):
    # Create Document — Topic Matrix
    lda_output = best_lda_model.transform(tfs)
    # column names
    topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]
    # index names
#     docnames = ["Doc" + str(i) for i in range(len(data))]
    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic
    
    print('df_document_topics: ',df_document_topic)

    # Topic-Keyword Matrix
    df_topic_keywords = pd.DataFrame(best_lda_model.components_)
    # Assign Column and Index
    df_topic_keywords.columns = feature_names
    df_topic_keywords.index = topicnames
    
    df_document_topic.to_csv('doc_topic.csv')
    df_topic_keywords.to_csv('topic_keywords.csv')
    # View
    print(df_topic_keywords.head())
    
    return df_document_topic,topicnames,lda_output

In [10]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anchi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [9]:
pip install gensim

  Stored in directory: C:\Users\anchi\AppData\Local\pip\Cache\wheels\ab\10\93\5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open
Note: you may need to restart the kernel to use updated packages.


In [11]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [12]:
doc_sample = df_csv[df_csv['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

NameError: name 'df_csv' is not defined